In [2]:
import os
import numpy as np
import librosa
import sounddevice as sd
import moviepy as mp

In [9]:
def extract_audio_from_video(video_path, save_flag=False, output_path=None):
    """
    Extracts the audio track from a video and returns it as a NumPy array with sample rate.
    """
    print(f"Extracting audio from {video_path}...")
    temp_audio_path = "temp_extracted_audio.wav"
    video = mp.VideoFileClip(video_path)
    if save_flag:
        if output_path is None:
            raise ValueError("Output path must be provided if save_flag is True.")
        video.audio.write_audiofile(output_path)  # removed verbose/logger
        y, sr = librosa.load(output_path, sr=None)
    else:
        video.audio.write_audiofile(temp_audio_path)  # removed verbose/logger
        y, sr = librosa.load(temp_audio_path, sr=None)
        os.remove(temp_audio_path)
    return y, sr

def load_audio(audio_path):
    """
    Loads an audio file (wav, mp3, etc.) and returns it as a NumPy array with sample rate.
    """
    print(f"Loading audio from {audio_path}...")
    y, sr = librosa.load(audio_path, sr=None)
    return y, sr

def extract_time_window(audio_array, sr, start_time, end_time):
    """
    Extracts a portion of the audio trace between start_time and end_time (in seconds).
    """
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    return audio_array[start_sample:end_sample]

def slow_down_audio(audio_segment, slowdown_factor):
    """
    Slows down an audio segment without changing its pitch using librosa.
    """
    print(f"Slowing down audio by a factor of {slowdown_factor} (without pitch change)...")
    return librosa.effects.time_stretch(audio_segment, rate=1.0 / slowdown_factor)

def play_audio_loop(audio_array, sr, nloops:int=1):
    """
    Plays the given audio array in an infinite loop. Press Ctrl+C to stop.
    """
    print("Playing slowed audio in loop. Press Ctrl+C to stop.")
    try:
        n = 1
        while n <= nloops:
            sd.play(audio_array, sr)
            sd.wait()
            n += 1
    except KeyboardInterrupt:
        print("\nLoop playback stopped.")

In [10]:
# Parameters
input_file = "./data/Moments_notice_rosenwinkel.mp3"         # or "input.wav" or "input.mp3"
start_time = 20.0                 # in seconds
end_time = 25.0                  # in seconds
slowdown_factor = 2.0           # 2x slower

In [5]:
# Step 1: Load audio trace
if input_file.lower().endswith((".mp4", ".mov", ".avi", ".mkv")):
    audio, sr = extract_audio_from_video(input_file, save_flag=True, output_path="./data/Moments_notice_rosenwinkel.mp3")
else:
    audio, sr = load_audio(input_file)

Loading audio from ./data/Moments_notice_rosenwinkel.mp3...


In [6]:
# Step 2: Extract time window
audio_segment = extract_time_window(audio, sr, start_time, end_time)

In [11]:
# Step 3: Slow down the segment
slowed_segment = slow_down_audio(audio_segment, slowdown_factor)

Slowing down audio by a factor of 2.0 (without pitch change)...


In [12]:
# Step 4: Play the slowed segment in loop
play_audio_loop(slowed_segment, sr)

Playing slowed audio in loop. Press Ctrl+C to stop.
